In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Load Data

In [ ]:
root_dir = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"

In [ ]:
folders = os.listdir(root_dir)
folders

In [ ]:
folders = ["benign", "malignant", "normal"]

In [ ]:
image_paths, labels, mask_paths = [], [], []

for folder in folders:
    files = os.listdir(os.path.join(root_dir, folder))
    for file in files:
        if "mask_" in file:
            continue
        elif "mask" in file:
            mask_paths.append(os.path.join(root_dir, folder, file))
        else:
            image_paths.append(os.path.join(root_dir, folder, file))
            labels.append(folder)

In [ ]:
df = pd.DataFrame({
    "image_path": image_paths,
    "mask_path": mask_paths,
    "label": labels
})

In [ ]:
df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
def load_image(path):
    img = load_img(path, target_size=(128, 128), color_mode='grayscale')
    image_array = img_to_array(img)
    image_array /= 255.
    return image_array

In [ ]:
images = []
masks = []

for image_path, mask_path in tqdm(zip(image_paths, mask_paths)):
    image = load_image(image_path)
    images.append(image)

    mask = load_image(mask_path)
    masks.append(mask)

In [ ]:
images = np.array(images)
masks = np.array(masks)

# Visualization

In [ ]:
def plot_image_and_mask(images, masks):
    plt.figure(figsize=(20, 6))

    for i in range(8):
        idx = np.random.randint(len(images))
        plt.subplot(3, 8, i + 1)
        plt.imshow(images[idx], cmap="gray")
        plt.axis("off")
        plt.title(image_paths[idx].split("/")[-1])

        plt.subplot(3, 8, i + 9)
        plt.imshow(masks[idx], cmap="gray")
        plt.axis("off")
        plt.title("Mask")

    plt.show()

In [ ]:
plot_image_and_mask(images, masks)

# Model

In [ ]:
# Drop "normal"
images = images[:-133]
masks = masks[:-133]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.1, random_state=42)

In [ ]:
def conv_block(input_layer, num_filters, drop_rate):
    conv = layers.Conv2D(num_filters, (3, 3), activation="elu", padding="same", kernel_initializer="he_normal")(input_layer)
    conv = layers.Dropout((drop_rate))(conv)
    conv = layers.Conv2D(num_filters, (3, 3), activation="elu", padding="same", kernel_initializer="he_normal")(conv)
    return conv

In [ ]:
def encoder_block(input_layer, num_filters, drop_rate):
    conv = conv_block(input_layer, num_filters, drop_rate)
    maxp = layers.MaxPooling2D((2, 2))(conv)
    return conv, maxp

In [ ]:
def decoder_block(input_layer, skip_features, drop_rate, num_filters):
    convt = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_layer)
    cnt = layers.concatenate([convt, skip_features])
    conv = conv_block(cnt, num_filters, drop_rate)
    return conv

In [ ]:
input_layer = layers.Input(shape=(128, 128, 1))

s1, p1 = encoder_block(input_layer, 32, 0.1)
s2, p2 = encoder_block(p1, 64, 0.1)
s3, p3 = encoder_block(p2, 128, 0.2)
s4, p4 = encoder_block(p3, 256, 0.2)
s5, p5 = encoder_block(p4, 512, 0.3)

bottle = conv_block(p5, 1024, 0.3)

d1 = decoder_block(bottle, s5, 0.3, 512)
d2 = decoder_block(d1, s4, 0.2, 256)
d3 = decoder_block(d2, s3, 0.2, 128)
d4 = decoder_block(d3, s2, 0.1, 64)
d5 = decoder_block(d4, s1, 0.1, 32)

output_layer = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d5)

model = models.Model(input_layer, output_layer)

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
plot_model(model)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

# Results

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["train", "valid"])
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["train", "valid"])
plt.show()

# Test

In [ ]:
preds = model.predict(X_test)

In [ ]:
def plot_predictions(preds):
    plt.figure(figsize=(20, 6))

    for i in range(8):
        idx = np.random.randint(len(preds))
        plt.subplot(3, 8, i + 1)
        plt.imshow(X_test[idx], cmap="gray")
        plt.axis("off")
        plt.title("Image")

        plt.subplot(3, 8, i + 9)
        plt.imshow(y_test[idx], cmap="gray")
        plt.axis("off")
        plt.title("Original Mask")

        plt.subplot(3, 8, i + 17)
        plt.imshow(preds[idx], cmap="gray")
        plt.axis("off")
        plt.title("Predicted Mask")

    plt.show()

In [ ]:
plot_predictions(preds)